In [1]:
import pandas as pd
import numpy as np
import string
import re
import random
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer


In [2]:
df=pd.read_csv(r"D:\Capstone\Datasets\Data_preparation\preprocessed_labels3_data.csv")


In [3]:
df.head(5)

,docid,text,topics,count_topics,text_expanded,text_wo_punct,remove_titles,text_tokenized,text_wo_stopwords,text_lemmatized,processed_text
0,2289,CompuServe Corp. Tuesday reported a surprising...,"['PERFORMANCE', 'ACCOUNTS/EARNINGS', 'CORPORAT...",3,CompuServe Corp. Tuesday reported a surprising...,compuserve corp tuesday reported a surprisingl...,compuserve corp tuesday reported a surprisingl...,"['compuserve', 'corp', 'tuesday', 'reported', ...","['compuserve', 'corp', 'tuesday', 'reported', ...","['compuserve', 'corp', 'tuesday', 'report', 's...",compuserve corp tuesday report surprisingly la...
1,2290,If dining at Planet Hollywood made you feel li...,"['STRATEGY/PLANS', 'NEW_PRODUCTS/SERVICES', 'C...",3,If dining at Planet Hollywood made you feel li...,if dining at planet hollywood made you feel li...,if dining at planet hollywood made you feel li...,"['if', 'dining', 'at', 'planet', 'hollywood', ...","['dining', 'planet', 'hollywood', 'made', 'fee...","['din', 'planet', 'hollywood', 'make', 'feel',...",din planet hollywood make feel like movie star...
2,2302,Columbia/HCA Healthcare Corp.'s proposed $299....,"['OWNERSHIP_CHANGES', 'MERGERS/ACQUISITIONS', ...",3,Columbia/HCA Healthcare Corp.'s proposed $299....,columbiahca healthcare corps proposed million...,columbiahca healthcare corps proposed million...,"['columbiahca', 'healthcare', 'corps', 'propos...","['columbiahca', 'healthcare', 'corps', 'propos...","['columbiahca', 'healthcare', 'corps', 'propos...",columbiahca healthcare corps propose million a...
3,2307,World oil prices slipped on Tuesday in a marke...,"['COMMODITY_MARKETS', 'ENERGY_MARKETS', 'MARKE...",3,World oil prices slipped on Tuesday in a marke...,world oil prices slipped on tuesday in a marke...,world oil prices slipped on tuesday in a marke...,"['world', 'oil', 'prices', 'slipped', 'on', 't...","['world', 'oil', 'prices', 'slipped', 'tuesday...","['world', 'oil', 'price', 'slip', 'tuesday', '...",world oil price slip tuesday market refiners s...
4,2312,Gulf War hero Colin Powell lent his prestige a...,"['GOVERNMENT/SOCIAL', 'DOMESTIC_POLITICS', 'EL...",3,Gulf War hero Colin Powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,gulf war hero colin powell lent his prestige a...,"['gulf', 'war', 'hero', 'colin', 'powell', 'le...","['gulf', 'war', 'hero', 'colin', 'powell', 'le...","['gulf', 'war', 'hero', 'colin', 'powell', 'le...",gulf war hero colin powell lend prestige popul...


In [4]:
df['topics'].value_counts().head(10)

topics
['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']             53939
['PERFORMANCE', 'ACCOUNTS/EARNINGS', 'CORPORATE/INDUSTRIAL']             50740
['COMMODITY_MARKETS', 'SOFT_COMMODITIES', 'MARKETS']                     36090
['OWNERSHIP_CHANGES', 'MERGERS/ACQUISITIONS', 'CORPORATE/INDUSTRIAL']    23106
['MONEY_MARKETS', 'INTERBANK_MARKETS', 'MARKETS']                        20439
['GOVERNMENT_FINANCE', 'GOVERNMENT_BORROWING', 'ECONOMICS']              17671
['COMMODITY_MARKETS', 'ENERGY_MARKETS', 'MARKETS']                       17271
['MONEY_MARKETS', 'FOREX_MARKETS', 'MARKETS']                            15811
['FUNDING/CAPITAL', 'BONDS/DEBT_ISSUES', 'CORPORATE/INDUSTRIAL']          9489
['COMMODITY_MARKETS', 'METALS_TRADING', 'MARKETS']                        7750
Name: count, dtype: int64

In [5]:

top_52_topics = df['topics'].value_counts().head(52)

# Create an empty DataFrame to store sampled data
sampled_data = pd.DataFrame(columns=df.columns)

for topic, count in top_52_topics.items():
    # Filter rows for the current topic
    topic_data = df[df['topics'] == topic]
    
    # If there are more than 500 samples, randomly select 500
    if count > 500:
        topic_data = topic_data.sample(n=500, random_state=42)
    
    # Append selected samples to the sampled_data DataFrame
    sampled_data = pd.concat([sampled_data, topic_data])

# Reset index of the sampled data
sampled_data.reset_index(drop=True, inplace=True)

# Print the shape of the sampled data
print("Shape of sampled data:", sampled_data.shape)


Shape of sampled data: (26000, 11)


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

# Count Vectorizer
vect = CountVectorizer()  
vects = vect.fit_transform(sampled_data.processed_text)
vects.shape

(26000, 71994)

In [7]:
tdm = vects.T

term_document_matrix = pd.DataFrame.sparse.from_spmatrix(tdm, index=vect.get_feature_names_out(), columns=sampled_data['docid'].astype(str))

print(term_document_matrix.shape)

(71994, 26000)


## Max Normalization

In [8]:
import numpy as np

max_values = term_document_matrix.max(axis=0)
normalized_term_document_matrix =np.divide(term_document_matrix, max_values)
normalized_term_document_matrix=normalized_term_document_matrix.values


## Truncate_SVD

In [9]:
%%time

from sklearn.decomposition import TruncatedSVD
import numpy as np

k =400 
svd = TruncatedSVD(n_components=k)

U = svd.fit_transform(normalized_term_document_matrix)
S = np.diag(svd.singular_values_)  
VT = svd.components_


S_inv = np.linalg.inv(S)

print("Inverse of S:")
print(S_inv)

Inverse of S:
[[0.00720928 0.         0.         ... 0.         0.         0.        ]
 [0.         0.0157339  0.         ... 0.         0.         0.        ]
 [0.         0.         0.0207617  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.11921289 0.         0.        ]
 [0.         0.         0.         ... 0.         0.11988585 0.        ]
 [0.         0.         0.         ... 0.         0.         0.12023337]]
CPU times: total: 13min 19s
Wall time: 12min 24s


In [10]:
Xq_prime = normalized_term_document_matrix.T

print("Xq_prime shape: ",Xq_prime.shape)
print("U shape: ",U.shape)
print("S_inv shape: ",S_inv.shape)


Xq_prime shape:  (26000, 71994)
U shape:  (71994, 400)
S_inv shape:  (400, 400)


In [11]:
Dq = Xq_prime.dot(U).dot(S_inv)
Dq.shape

(26000, 400)

In [12]:
sampled_data = sampled_data.reset_index(drop=True)
sampled_data.shape

(26000, 11)

In [13]:
sampled_data['topics'][9]

"['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']"

In [14]:
import ast

# Convert string representation of list to actual list
sampled_data['topics'] = sampled_data['topics'].apply(ast.literal_eval)

# Now the 'topics' column will contain lists instead of strings
print(sampled_data['topics'][9])  # Print the value at index 9 to verify


['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']


In [15]:
print(sampled_data['topics'][0]) 

['PERFORMANCE', 'COMMENT/FORECASTS', 'CORPORATE/INDUSTRIAL']


In [16]:
from sklearn.preprocessing import MultiLabelBinarizer

# Instantiate the binarizer
mlb = MultiLabelBinarizer()

# Fit and transform the 'topics' column of the DataFrame
binary_labels = mlb.fit_transform(sampled_data['topics'])

# Create a new DataFrame from the binary labels with appropriate column names
mlb_df = pd.DataFrame(binary_labels, columns=mlb.classes_, index=sampled_data.index)
mlb_df.head(3)


,ACCOUNTS/EARNINGS,ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,CONSUMER_FINANCE,CONSUMER_PRICES,...,REGULATION/POLICY,RESERVES,RETAIL_SALES,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
mlb.classes_

array(['ACCOUNTS/EARNINGS', 'ASSET_TRANSFERS', 'BALANCE_OF_PAYMENTS',
       'BIOGRAPHIES,_PERSONALITIES,_PEOPLE', 'BONDS/DEBT_ISSUES',
       'CAPACITY/FACILITIES', 'COMMENT/FORECASTS', 'COMMODITY_MARKETS',
       'CONSUMER_FINANCE', 'CONSUMER_PRICES', 'CONTRACTS/ORDERS',
       'CORPORATE/INDUSTRIAL', 'CREDIT_RATINGS', 'CRIME,_LAW_ENFORCEMENT',
       'DEFENCE', 'DISASTERS_AND_ACCIDENTS', 'DOMESTIC_POLITICS',
       'ECONOMICS', 'ECONOMIC_PERFORMANCE', 'ELECTIONS', 'ENERGY_MARKETS',
       'ENVIRONMENT_AND_NATURAL_WORLD', 'EUROPEAN_COMMUNITY',
       'EXPENDITURE/REVENUE', 'EXTERNAL_MARKETS', 'FOREX_MARKETS',
       'FUNDING/CAPITAL', 'GOVERNMENT/SOCIAL', 'GOVERNMENT_BORROWING',
       'GOVERNMENT_FINANCE', 'INDUSTRIAL_PRODUCTION', 'INFLATION/PRICES',
       'INTERBANK_MARKETS', 'INTERNATIONAL_RELATIONS', 'LOANS/CREDITS',
       'MANAGEMENT', 'MANAGEMENT_MOVES', 'MARKETS', 'MARKETS/MARKETING',
       'MERCHANDISE_TRADE', 'MERGERS/ACQUISITIONS', 'METALS_TRADING',
       'MONETARY/ECON

In [18]:
Dq.shape

(26000, 400)

In [19]:
mlb_df.shape

(26000, 61)

In [20]:
mlb_df

,ACCOUNTS/EARNINGS,ASSET_TRANSFERS,BALANCE_OF_PAYMENTS,"BIOGRAPHIES,_PERSONALITIES,_PEOPLE",BONDS/DEBT_ISSUES,CAPACITY/FACILITIES,COMMENT/FORECASTS,COMMODITY_MARKETS,CONSUMER_FINANCE,CONSUMER_PRICES,...,REGULATION/POLICY,RESERVES,RETAIL_SALES,SHARE_CAPITAL,SOFT_COMMODITIES,STRATEGY/PLANS,TRADE/RESERVES,"WAR,_CIVIL_WAR",WEATHER,WHOLESALE_PRICES
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model

In [21]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, f1_score
import numpy as np
import matplotlib.pyplot as plt

# Split data into train+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(Dq, mlb_df, test_size=0.2, random_state=42)

# Further split train+validation into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

num_classes = y_train.shape[1]

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model using the training set and validate it using the validation set
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=0)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy*100:.2f}%')

# Generate predictions
predicted_probs_test = model.predict(X_test)
predicted_labels_test = (predicted_probs_test > 0.3).astype(int)

# Generate classification report for the test set
test_report = classification_report(y_test, predicted_labels_test, target_names=mlb.classes_, output_dict=True)
print("\nTest Set Classification Report:\n", classification_report(y_test, predicted_labels_test, target_names=mlb.classes_))

# Print F1 scores for the test set
print(f"Test Set Macro-average F1 score: {test_report['macro avg']['f1-score']:.2f}")
print(f"Test Set Micro-average F1 score: {test_report['micro avg']['f1-score']:.2f}")
print(f"Test Set Weighted-average F1 score: {test_report['weighted avg']['f1-score']:.2f}")


Test accuracy: 42.63%
163/163 [==============================] - 0s 1ms/step

Test Set Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.90      0.89      0.90        91
                   ASSET_TRANSFERS       0.61      0.74      0.67       113
               BALANCE_OF_PAYMENTS       0.81      0.87      0.84       109
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       0.80      0.85      0.82        91
                 BONDS/DEBT_ISSUES       0.82      0.85      0.84       110
               CAPACITY/FACILITIES       0.74      0.77      0.75       412
                 COMMENT/FORECASTS       0.81      0.76      0.78       100
                 COMMODITY_MARKETS       0.93      0.96      0.94       290
                  CONSUMER_FINANCE       0.97      0.98      0.97        94
                   CONSUMER_PRICES       0.85      0.89      0.87        92
                  CONTRACTS/ORDERS       0.60      0

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# Assuming Dq is your features and mlb_df is your encoded labels
X = Dq
y = mlb_df.values

# Define the number of folds for cross-validation
n_splits = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Placeholder for aggregated test set results
y_test_all = []
predicted_labels_all = []

for train_index, test_index in kf.split(X, y.argmax(1)):  # Using argmax to simulate stratification
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    num_classes = y_train.shape[1]
    
    # Define the model architecture
    model = Sequential([
        Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='sigmoid')  # Sigmoid activation for multilabel classification
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Define early stopping criteria
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model with early stopping
    model.fit(X_train, y_train, epochs=100, batch_size=64, 
              validation_split=0.2, callbacks=[early_stopping], verbose=0)
    
    # Generate predictions for this fold's test set
    predicted_probs = model.predict(X_test)
    predicted_labels = (predicted_probs > 0.5).astype(int)
    
    # Aggregate the results
    y_test_all.append(y_test)
    predicted_labels_all.append(predicted_labels)

# Concatenate all test set results
y_test_all = np.concatenate(y_test_all, axis=0)
predicted_labels_all = np.concatenate(predicted_labels_all, axis=0)

# Generate and print the overall classification report
report = classification_report(y_test_all, predicted_labels_all, target_names=mlb.classes_)
print("Overall Classification Report:\n", report)


82/82 [==============================] - 0s 2ms/step
Overall Classification Report:
                                     precision    recall  f1-score   support

                 ACCOUNTS/EARNINGS       0.97      0.60      0.74       500
                   ASSET_TRANSFERS       0.00      0.00      0.00       500
               BALANCE_OF_PAYMENTS       0.92      0.07      0.13       500
BIOGRAPHIES,_PERSONALITIES,_PEOPLE       1.00      0.01      0.01       500
                 BONDS/DEBT_ISSUES       0.94      0.47      0.63       500
               CAPACITY/FACILITIES       0.74      0.55      0.63      2000
                 COMMENT/FORECASTS       1.00      0.00      0.00       500
                 COMMODITY_MARKETS       0.97      0.85      0.91      1500
                  CONSUMER_FINANCE       1.00      0.01      0.02       500
                   CONSUMER_PRICES       0.81      0.41      0.55       500
                  CONTRACTS/ORDERS       0.00      0.00      0.00       500
  

C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\gayat\.conda\envs\pracenv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
